In [1]:
%load_ext autoreload
%autoreload 2

## Import Dependencies

In [2]:

import comet_ml

import sys
sys.path.append("../modules")

from training.train import train

/home/pauliusztin/anaconda3/envs/hands-on-llms/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Train Model

In [3]:
trainer, training_dataset, validation_dataset = train(debug=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.39s/it]
/home/pauliusztin/anaconda3/envs/hands-on-llms/lib/python3.9/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/iusztinpaul/hands-on-llms/049725e75c0a45edaba52f6f9ff51dfd

Token indices sequence length is longer than the specified maximum sequence length for this model (2302 > 2048). Running this sequence through the model will result in indexing errors
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.47 GiB (GPU 0; 11.76 GiB total capacity; 6.23 GiB already allocated; 2.98 GiB free; 8.56 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.state.log_history

[{'loss': 1.0479, 'learning_rate': 0.0002, 'epoch': 1.0, 'step': 1},
 {'loss': 1.0542, 'learning_rate': 0.0002, 'epoch': 2.0, 'step': 2},
 {'loss': 1.0474, 'learning_rate': 0.0002, 'epoch': 3.0, 'step': 3},
 {'eval_loss': 2.1510212421417236,
  'eval_perplexity': 6.727502466219448e-08,
  'eval_runtime': 5.4704,
  'eval_samples_per_second': 1.097,
  'eval_steps_per_second': 0.366,
  'epoch': 3.0,
  'step': 3},
 {'loss': 1.0546, 'learning_rate': 0.0002, 'epoch': 4.0, 'step': 4},
 {'loss': 1.0475, 'learning_rate': 0.0002, 'epoch': 5.0, 'step': 5},
 {'loss': 1.0368, 'learning_rate': 0.0002, 'epoch': 6.0, 'step': 6},
 {'eval_loss': 2.1235926151275635,
  'eval_perplexity': 6.37254586877134e-08,
  'eval_runtime': 5.4891,
  'eval_samples_per_second': 1.093,
  'eval_steps_per_second': 0.364,
  'epoch': 6.0,
  'step': 6},
 {'train_runtime': 135.3107,
  'train_samples_per_second': 0.399,
  'train_steps_per_second': 0.044,
  'total_flos': 982691063267328.0,
  'train_loss': 1.0480629007021587,
  'ep

## Test Model

In [ ]:
# test_dataset = finqa.FinQADataset(
#     data_path=ROOT_DATASET_DIR / "private_test.json", scope=constants.Scope.TESTING
# ).to_huggingface()
# test_dataset

In [ ]:
# test_dataset["text"][0]

In [ ]:
# loaded_model, loaded_tokenizer, _ = models.build_qlora_model(
#     model_id="tiiuae/falcon-7b-instruct", peft_model_id=RESULT_DIR / "checkpoint-280"
# )

In [ ]:
# print(
#     models.prompt(
#         model=loaded_model,
#         tokenizer=loaded_tokenizer,
#         input_text=test_dataset["text"][0],
#         max_new_tokens=100
#     )
# )